<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Reservoir%20Simulation%20Ertekin/Unit%205%20Finite-Difference%20Approximation%20to%20Linear-Flow%20Equations/fd1d_linearflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 5 Finite-Difference Approximations to Linear-Flow Problems**

## Explicit Formulation

$$p_{i}^{n+1} = (\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i \cdot q_{lsc_i} + (\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i \cdot [T_{lx_{i+1/2}}^{n} p_{i+1}^{n}-(T_{lx_{i+1/2}}^{n} + T_{lx_{i-1/2}}^{n})p_{i}^{n}+T_{lx_{i-1/2}}p_{i-1}^{n}]$$

In the following codes, $(\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i$ will be written as `factor`

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!git clone https://github.com/yohanesnuwara/reservoir-engineering

Cloning into 'reservoir-engineering'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 845 (delta 49), reused 0 (delta 0), pack-reused 696
Receiving objects: 100% (845/845), 4.95 MiB | 30.33 MiB/s, done.
Resolving deltas: 100% (347/347), done.


## Example 5.8 

![Example 5 8](https://user-images.githubusercontent.com/51282928/75949840-09e36380-5eda-11ea-824c-dd5c9ac6f6d5.PNG)

In [0]:
"Task. determine pressure distribution during first year of production with timestep 10 days"

# known
pi = 6000 # initial pressure, in psia
delta_x = 1000 # ft
delta_y = 1000
delta_z = 75
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 3.5E-06 # phase compressibility, psi^-1
k_x = 15 # perm in x direction, md
poro = 0.18
mu = 10 # phase viscosity, cp

delta_t = 10 # days
qsc = -150 # minus means production, stb/d

# calculate 